# Reshape Fuel Prices - Duke Energy Progress

3/18/2021 \
by [Mauricio Hernandez](mmh54@duke.edu)

In [1]:
import csv
import datetime as dt
import numpy as np
import pandas as pd

In [4]:
df_lookup = pd.read_csv('./inputs/UnitLookupAndDetailTable_(DEC-DEP).csv')
df_fuel_DEP = pd.read_csv('./inputs/UNIT_FUEL_PRICE(DEP 2019).csv')
list(df_fuel_DEP.columns)

['REGION',
 'UNIT_NAME',
 'CC_KEY',
 'FUEL_CONTRACT',
 'FUEL_TYPE',
 'EDITION',
 'PRICE $/MBTU',
 'FROM_DATE',
 'TO_DATE',
 'MODIFIED']

In [5]:
#Slicing data and filter all the values where end date is before Jan 1st
df_fuel_DEP['UNIT_ID'] = df_fuel_DEP.UNIT_NAME + '_'+ df_fuel_DEP.CC_KEY.apply(str)
df_fuel_DEP = df_fuel_DEP.loc[:, ['UNIT_ID', 'FUEL_TYPE','PRICE $/MBTU', 'FROM_DATE', 'TO_DATE']]

df_fuel_DEP.sort_values(by=['UNIT_ID', 'FUEL_TYPE'], inplace=True)

df_fuel_DEP.to_csv('./outputs/UNIT_FUEL_PRICE(DEP 2019)_sorted.csv', sep=',', encoding='utf-8', index= False)

df_fuel_DEP.head()

,UNIT_ID,FUEL_TYPE,PRICE $/MBTU,FROM_DATE,TO_DATE
0,ASHV_CC01_1,LOIL,14.214,12/5/2019,12/6/2019
1,ASHV_CC01_1,LOIL,14.397,12/6/2019,12/11/2019
2,ASHV_CC01_1,LOIL,14.551,12/11/2019,12/13/2019
3,ASHV_CC01_1,LOIL,14.462,12/13/2019,12/14/2019
4,ASHV_CC01_1,LOIL,14.542,12/14/2019,12/15/2019


## Descriptive statistics

Data from Duke Energy Carolinas and Duke Energy Progress

In [6]:
df_fuel_DEP.describe(include='all')

,UNIT_ID,FUEL_TYPE,PRICE $/MBTU,FROM_DATE,TO_DATE
count,35155,35155,35155.00000,35155,35155
unique,74,3,NaN,600,601
top,DCEP_CT01_0,NGAS,NaN,3/9/2019,8/10/2019
freq,584,17542,NaN,79,79
mean,NaN,NaN,10.95653,NaN,NaN
std,NaN,NaN,16.12099,NaN,NaN
min,NaN,NaN,1.75900,NaN,NaN
25%,NaN,NaN,2.66300,NaN,NaN
50%,NaN,NaN,4.79800,NaN,NaN
75%,NaN,NaN,14.51300,NaN,NaN


### Calculating range of days between initial and end dates

In [7]:
def convertStringToDate(date_string): 
    date_obj = dt.datetime.strptime(date_string.split(" ")[0], '%m/%d/%Y')
    #if date_obj - dt.date(2018, 7, 11)
    return date_obj    

#convertStringToDate('5/10/2018')
df_fuel_DEP['FROM_DATE'] = df_fuel_DEP['FROM_DATE'].apply(convertStringToDate)
df_fuel_DEP['TO_DATE'] = df_fuel_DEP['TO_DATE'].apply(convertStringToDate)

df_fuel_DEP.describe(include='all')

,UNIT_ID,FUEL_TYPE,PRICE $/MBTU,FROM_DATE,TO_DATE
count,35155,35155,35155.00000,35155,35155
unique,74,3,NaN,358,362
top,DCEP_CT01_0,NGAS,NaN,2019-10-12 00:00:00,2019-10-05 00:00:00
freq,584,17542,NaN,133,133
first,NaN,NaN,NaN,2018-12-22 00:00:00,2018-12-23 00:00:00
last,NaN,NaN,NaN,2019-12-31 00:00:00,2020-01-18 00:00:00
mean,NaN,NaN,10.95653,NaN,NaN
std,NaN,NaN,16.12099,NaN,NaN
min,NaN,NaN,1.75900,NaN,NaN
25%,NaN,NaN,2.66300,NaN,NaN


In [8]:
First_day = convertStringToDate('1/1/2019')
Last_day = convertStringToDate('12/31/2019')

#remove all the values where the end dates are in 2018
df_fuel_DEP['END_YEAR'] = df_fuel_DEP['TO_DATE'].map(lambda TO_DATE: TO_DATE.year)
df_fuel_DEP['START_YEAR'] = df_fuel_DEP['FROM_DATE'].map(lambda FROM_DATE: FROM_DATE.year)

df_fuel_DEP = df_fuel_DEP[df_fuel_DEP['START_YEAR'] < 2020]
df_fuel_DEP = df_fuel_DEP[df_fuel_DEP['END_YEAR'] >= 2019]

df_fuel_DEP['FROM_DATE'] = df_fuel_DEP['FROM_DATE'].map(lambda FROM_DATE: First_day if (First_day - FROM_DATE).days > 0 else FROM_DATE )
df_fuel_DEP['TO_DATE'] = df_fuel_DEP['TO_DATE'].map(lambda TO_DATE: Last_day if (TO_DATE - Last_day).days > 0 else TO_DATE)

df_fuel_DEP = df_fuel_DEP[df_fuel_DEP['TO_DATE'] != First_day]

df_fuel_DEP.describe(include='all')

,UNIT_ID,FUEL_TYPE,PRICE $/MBTU,FROM_DATE,TO_DATE,END_YEAR,START_YEAR
count,34584,34584,34584.000000,34584,34584,34584.000000,34584.000000
unique,74,3,NaN,349,348,NaN,NaN
top,DCEP_CT01_0,NGAS,NaN,2019-09-21 00:00:00,2019-12-31 00:00:00,NaN,NaN
freq,574,17178,NaN,133,190,NaN,NaN
first,NaN,NaN,NaN,2019-01-01 00:00:00,2019-01-02 00:00:00,NaN,NaN
last,NaN,NaN,NaN,2019-12-31 00:00:00,2019-12-31 00:00:00,NaN,NaN
mean,NaN,NaN,10.992740,NaN,NaN,2019.003932,2018.999884
std,NaN,NaN,16.152249,NaN,NaN,0.062587,0.010754
min,NaN,NaN,1.759000,NaN,NaN,2019.000000,2018.000000
25%,NaN,NaN,2.656000,NaN,NaN,2019.000000,2019.000000


In [9]:
# Adding columns to compute number of days from FROM_DATE to TO_DATE 
df_fuel_DEP['DAYS'] = df_fuel_DEP['TO_DATE'] - df_fuel_DEP['FROM_DATE']
df_fuel_DEP['DAYS'] = df_fuel_DEP['DAYS'].map(lambda DAYS: DAYS.days )

df_fuel_DEP['REF_FROM_DATE'] = df_fuel_DEP['FROM_DATE'] - First_day
df_fuel_DEP['REF_FROM_DATE'] = df_fuel_DEP['REF_FROM_DATE'].map(lambda DAYS: DAYS.days )

# Replace last value when the number of days is zero
df_fuel_DEP['DAYS'] = np.where((df_fuel_DEP['DAYS'] == 0) & (df_fuel_DEP['TO_DATE'] == Last_day), 1, df_fuel_DEP['DAYS'])

df_fuel_DEP = df_fuel_DEP.loc[:, ['UNIT_ID', 'FUEL_TYPE', 'PRICE $/MBTU', 'FROM_DATE', 'TO_DATE', 'DAYS', 'REF_FROM_DATE']]
df_fuel_DEP.head()

,UNIT_ID,FUEL_TYPE,PRICE $/MBTU,FROM_DATE,TO_DATE,DAYS,REF_FROM_DATE
0,ASHV_CC01_1,LOIL,14.214,2019-12-05,2019-12-06,1,338
1,ASHV_CC01_1,LOIL,14.397,2019-12-06,2019-12-11,5,339
2,ASHV_CC01_1,LOIL,14.551,2019-12-11,2019-12-13,2,344
3,ASHV_CC01_1,LOIL,14.462,2019-12-13,2019-12-14,1,346
4,ASHV_CC01_1,LOIL,14.542,2019-12-14,2019-12-15,1,347


In [10]:
# Creating pivot tableto summarize unit units and fuel type
df_fuel_DEP_pivot = df_fuel_DEP.groupby(['UNIT_ID', 'FUEL_TYPE']).sum()
df_fuel_DEP_pivot.to_csv('./outputs/fuel_summary.csv', sep=',', encoding='utf-8')
#print(list(df_fuel_DEP_pivot.index))
df_fuel_DEP_pivot

PRICE $/MBTU  DAYS  REF_FROM_DATE
UNIT_ID     FUEL_TYPE                                   
ASHV_CC01_1 LOIL            101.256    26           2415
            NGAS            449.559   184          39679
ASHV_CC02_1 LOIL            101.256    26           2415
            NGAS            449.559   329          39679
ASHV_CT03_0 LOIL           3555.903   364          40786
...                             ...   ...            ...
WCOP_CT13_0 LOIL           3575.569   364          40786
            NGAS           1046.149   365          59620
WCOP_CT14_0 LOIL           3575.569   364          40786
            NGAS           1046.149   365          59620
WSPN_CT01_0 LOIL           3564.400   364          40786

[135 rows x 3 columns]

## Manipulating dataframe to organize data

In [11]:
First_day = convertStringToDate('1/1/2019')
Last_day = convertStringToDate('12/31/2019')

#Create list with dates from First_day to last_day
date_list = [First_day + dt.timedelta(days=x) for x in range(0, (Last_day-First_day).days + 1)]

date_str_list = []
for date in date_list:
    date_str_list.append(date.strftime("%m/%d/%Y"))

In [12]:
#create results dataframe to store prices every day
df_fuel_result = pd.DataFrame(index=df_fuel_DEP_pivot.index, columns=date_list)
#df_fuel_DEP_pivot = df_fuel_DEP_pivot.reindex(columns = df_fuel_DEP_pivot.columns.tolist() + date_str_list)
                                
df_fuel_result.head(n=5)

2019-01-01 2019-01-02 2019-01-03 2019-01-04 2019-01-05  \
UNIT_ID     FUEL_TYPE                                                          
ASHV_CC01_1 LOIL             NaN        NaN        NaN        NaN        NaN   
            NGAS             NaN        NaN        NaN        NaN        NaN   
ASHV_CC02_1 LOIL             NaN        NaN        NaN        NaN        NaN   
            NGAS             NaN        NaN        NaN        NaN        NaN   
ASHV_CT03_0 LOIL             NaN        NaN        NaN        NaN        NaN   

                      2019-01-06 2019-01-07 2019-01-08 2019-01-09 2019-01-10  \
UNIT_ID     FUEL_TYPE                                                          
ASHV_CC01_1 LOIL             NaN        NaN        NaN        NaN        NaN   
            NGAS             NaN        NaN        NaN        NaN        NaN   
ASHV_CC02_1 LOIL             NaN        NaN        NaN        NaN        NaN   
            NGAS             NaN        NaN        NaN        NaN        NaN   
ASHV_CT03_0 LOIL             NaN        NaN        NaN        NaN        NaN   

                       ... 2019-12-22 2019-12-23 2019-12-24 2019-12-25  \
UNIT_ID     FUEL_TYPE  ...                                               
ASHV_CC01_1 LOIL       ...        NaN        NaN        NaN        NaN   
            NGAS       ...        NaN        NaN        NaN        NaN   
ASHV_CC02_1 LOIL       ...        NaN        NaN        NaN        NaN   
            NGAS       ...        NaN        NaN        NaN        NaN   
ASHV_CT03_0 LOIL       ...        NaN        NaN        NaN        NaN   

                      2019-12-26 2019-12-27 2019-12-28 2019-12-29 2019-12-30  \
UNIT_ID     FUEL_TYPE                                                          
ASHV_CC01_1 LOIL             NaN        NaN        NaN        NaN        NaN   
            NGAS             NaN        NaN        NaN        NaN        NaN   
ASHV_CC02_1 LOIL             NaN        NaN        NaN        NaN        NaN   
            NGAS             NaN        NaN        NaN        NaN        NaN   
ASHV_CT03_0 LOIL             NaN        NaN        NaN        NaN        NaN   

                      2019-12-31  
UNIT_ID     FUEL_TYPE             
ASHV_CC01_1 LOIL             NaN  
            NGAS             NaN  
ASHV_CC02_1 LOIL             NaN  
            NGAS             NaN  
ASHV_CT03_0 LOIL             NaN  

[5 rows x 365 columns]

In [13]:
current_index = ()
old_index = ()
aux_index = 0
fuel_price_list = [None] * 365

for index, row in df_fuel_DEP.iterrows():
    aux_index = index
    index_current = (row['UNIT_ID'], row['FUEL_TYPE'])
    
    # access data using column names
    fuel_price = row['PRICE $/MBTU']
    days = row['DAYS']
    ref_day = row['REF_FROM_DATE']
    
    current_index = (row['UNIT_ID'], row['FUEL_TYPE'])
    #print(index, row['UNIT_ID'], row['FUEL_TYPE'], row['PRICE $/MBTU'], row['REF_FROM_DATE'], row['DAYS'])
        
    if index == 0:
        old_index = current_index
    
    if (old_index != current_index):
        df_fuel_result.loc[old_index] = fuel_price_list

        old_index = current_index
        fuel_price_list = [None] * 365
    
    fuel_price_list[ref_day:(ref_day + days)] = [fuel_price]*(days)

        #print(index, row['PRICE $/MBTU'], row['REF_FROM_DATE'], row['DAYS'])
#Save last value
if aux_index != 0 :
    df_fuel_result.loc[current_index] = fuel_price_list
    
df_fuel_result.head()

2019-01-01 2019-01-02 2019-01-03 2019-01-04 2019-01-05  \
UNIT_ID     FUEL_TYPE                                                          
ASHV_CC01_1 LOIL            None       None       None       None       None   
            NGAS            None       None       None       None       None   
ASHV_CC02_1 LOIL            None       None       None       None       None   
            NGAS            None       None       None       None       None   
ASHV_CT03_0 LOIL          12.444     12.444     12.468     12.468     12.758   

                      2019-01-06 2019-01-07 2019-01-08 2019-01-09 2019-01-10  \
UNIT_ID     FUEL_TYPE                                                          
ASHV_CC01_1 LOIL            None       None       None       None       None   
            NGAS            None       None       None       None       None   
ASHV_CC02_1 LOIL            None       None       None       None       None   
            NGAS            None       None       None       None       None   
ASHV_CT03_0 LOIL          13.036     13.036     13.287     13.287     13.287   

                       ... 2019-12-22 2019-12-23 2019-12-24 2019-12-25  \
UNIT_ID     FUEL_TYPE  ...                                               
ASHV_CC01_1 LOIL       ...     14.807     14.807     14.807     14.807   
            NGAS       ...      2.201      2.201      2.079      2.028   
ASHV_CC02_1 LOIL       ...     14.807     14.807     14.807     14.807   
            NGAS       ...         99         99         99         99   
ASHV_CT03_0 LOIL       ...     14.807     14.807     14.807     14.807   

                      2019-12-26 2019-12-27 2019-12-28 2019-12-29 2019-12-30  \
UNIT_ID     FUEL_TYPE                                                          
ASHV_CC01_1 LOIL          14.807     14.807     14.807     14.807     14.807   
            NGAS           2.063      2.028      1.759      1.759      1.759   
ASHV_CC02_1 LOIL          14.807     14.807     14.807     14.807     14.807   
            NGAS              99         99         99         99         99   
ASHV_CT03_0 LOIL          14.807     14.807     14.807     14.807     14.807   

                      2019-12-31  
UNIT_ID     FUEL_TYPE             
ASHV_CC01_1 LOIL            None  
            NGAS           2.109  
ASHV_CC02_1 LOIL            None  
            NGAS           2.109  
ASHV_CT03_0 LOIL            None  

[5 rows x 365 columns]

In [15]:
df_fuel_result.to_csv('./outputs/UNIT_FUEL_PRICES_DEP_Results.csv', sep=',', encoding='utf-8')
df_fuel_DEP.to_csv('./outputs/UNIT_FUEL_PRICES_DEP_Short.csv', sep=',', encoding='utf-8')

In [54]:
#dfSummary['UNIT_ID'] dfSummary.UNIT_ID == 'ALLE_UN01_0')
#dfSummary[dfSummary.DAYS == 364]

PRICE $/MBTU  END_YEAR  START_YEAR  DAYS
UNIT_ID     FUEL_TYPE                                          
ALLE_UN01_0 COAL           135.5794     88837       88835   364
            LOIL          3590.3080    496675      496674   364
ALLE_UN02_0 COAL           135.3128     88837       88835   364
            LOIL          3590.3080    496675      496674   364
ALLE_UN03_0 COAL           137.1085     88837       88835   364
...                             ...       ...         ...   ...
RCKN_CT03_0 NGAS           874.4140    658195      658194   364
RCKN_CT04_0 LOIL          3595.8000    496675      496674   364
            NGAS           874.4140    658195      658194   364
RCKN_CT05_0 LOIL          3595.8000    496675      496674   364
            NGAS           874.4140    658195      658194   364

[106 rows x 4 columns]